In [77]:
import warnings
warnings.filterwarnings('ignore')

In [78]:
from pathlib import Path
import pandas as pd
import requests
import json

from api_key import geoapify_key

In [79]:
# Create a file path for melbourne_listings
melb_data = Path('Resources/melbourne_listings.csv')
melb_full_df = pd.read_csv(melb_data)

# Read in the data
melb_full_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1044679652057774327,https://www.airbnb.com/rooms/1044679652057774327,20240316023834,2024-03-18,previous scrape,Jubilee park home,Great location:3mins walking to the large aqua...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,378158616,...,NaN,NaN,NaN,NaN,f,2,2,0,0,NaN
1,38971182,https://www.airbnb.com/rooms/38971182,20240316023834,2024-03-18,city scrape,Furnished accomodation,close to Hawthorn station,NaN,https://a0.muscache.com/pictures/95071a26-902a...,298550539,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,1036489765594430755,https://www.airbnb.com/rooms/1036489765594430755,20240316023834,2024-03-18,city scrape,Lovely Ringwood Studio w/ Ensuite,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,551026724,...,5.00,5.00,5.00,NaN,t,10,0,10,0,1.27
3,28139568,https://www.airbnb.com/rooms/28139568,20240316023834,2024-03-16,previous scrape,Close to Monash Caulfield Short lease!!!,NaN,NaN,https://a0.muscache.com/pictures/48b8ed68-d51c...,212558156,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
4,572431386115618941,https://www.airbnb.com/rooms/572431386115618941,20240316023834,2024-03-16,city scrape,Selina St Kilda - Bed in 4-Bed Dormitory Room,You'll enjoy your time at this cheerful getaway.,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,446636199,...,4.43,4.37,3.97,NaN,t,8,0,4,4,1.21


In [80]:
# Removing irrevalent columns, removed location data except for neighborhood_cleansed and lat, long. Removed host_since as will only look at past years data
melb_filtered_df = melb_full_df.drop(columns=["name", "listing_url", "scrape_id", "last_scraped", "source", "description", "neighborhood_overview", "picture_url", "host_url", "host_name", "host_about", "host_thumbnail_url", "host_picture_url", "host_neighbourhood", "host_listings_count", "host_total_listings_count", "host_verifications", "neighbourhood", "neighbourhood_group_cleansed", "calendar_updated", "calendar_last_scraped", "license", "instant_bookable", "minimum_nights", "maximum_nights", "minimum_minimum_nights", "minimum_maximum_nights", "maximum_minimum_nights", "maximum_maximum_nights", "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "has_availability", "availability_30", "availability_60", "availability_90", "availability_365", "instant_bookable"])
melb_filtered_df.head()

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1044679652057774327,378158616,2020-12-03,"Box Hill South, Australia",within an hour,100%,79%,f,t,t,...,NaN,NaN,NaN,NaN,NaN,2,2,0,0,NaN
1,38971182,298550539,2019-09-28,NaN,a few days or more,0%,0%,f,t,t,...,NaN,NaN,NaN,NaN,NaN,1,0,1,0,NaN
2,1036489765594430755,551026724,2023-12-14,NaN,NaN,NaN,NaN,NaN,t,t,...,5.0,5.0,5.00,5.00,5.00,10,0,10,0,1.27
3,28139568,212558156,2018-08-30,"Melbourne, Australia",NaN,NaN,NaN,f,t,f,...,NaN,NaN,NaN,NaN,NaN,1,0,1,0,NaN
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,4.0,4.6,4.43,4.37,3.97,8,0,4,4,1.21


In [81]:
#Removing rows with null values in review scores and host_reponse_time
melb_refined_df = melb_filtered_df.dropna()
#melb_refined_df

In [82]:
def get_distances (api_key, source_coords, target_coords):
    url = f"https://api.geoapify.com/v1/routematrix?apiKey={api_key}"
    headers = {"Content-Type": "application/json"}
    data = {
    "mode": "drive",
    "sources": [{"location": source_coords}],
    "targets": [{"location": target_coords}]
    }
    try:
        resp = requests.post(url, headers=headers, json=data)
        response_data = resp.json()
        distances = response_data["sources_to_targets"][0][0]["distance"]  # Assuming you want the first distance
        return distances
    except requests.exceptions.HTTPError as e:
        print(f"Error: {e.response.text}")
        return None

In [85]:
source_location = [144.9433,-37.8136] #melbourne coordinates
api_key = geoapify_key
distances_from_cbd = []
num=0
for index, row in melb_refined_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["latitude"]
    longitude = row["longitude"]
    target_location = [longitude,latitude]
    distance = get_distances(api_key, source_location, target_location)
    distances_from_cbd.append(distance)
    num=num +1
    print(f"request {num}")
    print(distance)
melb_refined_df["distances_from_cbd"] = distances_from_cbd
melb_refined_df

request 1
7434
request 2
10140
request 3
1451
request 4
24913
request 5
6951
request 6
3354
request 7
3066
request 8
9389
request 9
5728
request 10
19214
request 11
2008
request 12
12928
request 13
11678
request 14
15905
request 15
12532
request 16
38834
request 17
13034
request 18
51325
request 19
47194
request 20
22965
request 21
14241
request 22
57549
request 23
4186
request 24
11153
request 25
17845
request 26
2293
request 27
2865
request 28
7031
request 29
3852
request 30
15973
request 31
9932
request 32
20629
request 33
22007
request 34
8286
request 35
15578
request 36
28268
request 37
20904
request 38
6047
request 39
22536
request 40
4243
request 41
9976
request 42
7677
request 43
12461
request 44
12784
request 45
20777
request 46
15793
request 47
13137
request 48
33140
request 49
2540
request 50
1862
request 51
8732
request 52
7599
request 53
4668
request 54
8335
request 55
26101
request 56
11991
request 57
31851
request 58
3333
request 59
2631
request 60
60809
request 61
3070


,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distances_from_cbd
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,4.60,4.43,4.37,3.97,8,0,4,4,1.21,7434
11,727686708237430703,382295537,2020-12-30,"Melbourne, Australia",within a few hours,89%,57%,f,t,t,...,4.20,4.20,4.20,4.20,3,0,3,0,0.81,10140
16,1051913801382467131,22860147,2014-10-22,"Melbourne, Australia",within an hour,100%,99%,f,t,t,...,5.00,5.00,4.50,4.00,105,105,0,0,0.98,1451
17,1106234039745058155,16365362,2014-06-04,"Melbourne, Australia",within an hour,100%,100%,f,t,t,...,5.00,5.00,5.00,5.00,1,0,1,0,1.00,24913
18,1053974246558615906,551937839,2023-12-21,"Point Cook, Australia",within an hour,97%,94%,f,t,t,...,3.50,4.00,4.00,5.00,7,7,0,0,2.00,6951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25072,30836862,22548036,2014-10-14,"Melbourne, Australia",within a few hours,100%,95%,f,t,t,...,4.80,4.89,4.78,4.53,3,3,0,0,2.74,3351
25078,1061360537948958353,517641271,2023-06-01,"Melbourne, Australia",within an hour,89%,90%,f,t,t,...,3.33,3.67,5.00,3.67,11,11,0,0,3.00,2627
25081,710398956041051683,90729398,2016-08-19,"Victoria, Australia",within a few hours,82%,41%,f,t,t,...,5.00,5.00,5.00,3.00,339,75,262,2,0.08,3196
25082,925945938663212093,516470484,2023-05-25,"Melbourne, Australia",within an hour,100%,93%,t,t,t,...,4.59,4.96,4.98,4.84,5,5,0,0,6.09,1641


In [91]:
#convert distances from cbd to kilometers
melb_refined_df["distances_from_cbd_km"] = melb_refined_df["distances_from_cbd"] / 1000
melb_refined_df

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distances_from_cbd,distances_from_cbd_km
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,4.43,4.37,3.97,8,0,4,4,1.21,7434.0,7.434
11,727686708237430703,382295537,2020-12-30,"Melbourne, Australia",within a few hours,89%,57%,f,t,t,...,4.20,4.20,4.20,3,0,3,0,0.81,10140.0,10.140
16,1051913801382467131,22860147,2014-10-22,"Melbourne, Australia",within an hour,100%,99%,f,t,t,...,5.00,4.50,4.00,105,105,0,0,0.98,1451.0,1.451
17,1106234039745058155,16365362,2014-06-04,"Melbourne, Australia",within an hour,100%,100%,f,t,t,...,5.00,5.00,5.00,1,0,1,0,1.00,24913.0,24.913
18,1053974246558615906,551937839,2023-12-21,"Point Cook, Australia",within an hour,97%,94%,f,t,t,...,4.00,4.00,5.00,7,7,0,0,2.00,6951.0,6.951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25072,30836862,22548036,2014-10-14,"Melbourne, Australia",within a few hours,100%,95%,f,t,t,...,4.89,4.78,4.53,3,3,0,0,2.74,3351.0,3.351
25078,1061360537948958353,517641271,2023-06-01,"Melbourne, Australia",within an hour,89%,90%,f,t,t,...,3.67,5.00,3.67,11,11,0,0,3.00,2627.0,2.627
25081,710398956041051683,90729398,2016-08-19,"Victoria, Australia",within a few hours,82%,41%,f,t,t,...,5.00,5.00,3.00,339,75,262,2,0.08,3196.0,3.196
25082,925945938663212093,516470484,2023-05-25,"Melbourne, Australia",within an hour,100%,93%,t,t,t,...,4.96,4.98,4.84,5,5,0,0,6.09,1641.0,1.641


In [94]:
#categorise the distances from the cbd
distances_from_cbd_categories = []
for index, row in melb_refined_df.iterrows():
    distance_km = row["distances_from_cbd_km"]

    if distance_km < 5:
        category = "less than 5km"
    elif distance_km < 10:
        category = "less than 10km"  
    elif distance_km < 15:
        category = "less than 15km"            
    elif distance_km < 20:
        category = "less than 20km"  
    elif distance_km < 25:
        category = "less than 25km"  
    else:
        category = "greater than 25km"
        
    distances_from_cbd_categories.append(category)

melb_refined_df["distances_from_cbd_category"] = distances_from_cbd_categories
melb_refined_df

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distances_from_cbd,distances_from_cbd_km,distances_from_cbd_category
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,4.37,3.97,8,0,4,4,1.21,7434.0,7.434,less than 10km
11,727686708237430703,382295537,2020-12-30,"Melbourne, Australia",within a few hours,89%,57%,f,t,t,...,4.20,4.20,3,0,3,0,0.81,10140.0,10.140,less than 15km
16,1051913801382467131,22860147,2014-10-22,"Melbourne, Australia",within an hour,100%,99%,f,t,t,...,4.50,4.00,105,105,0,0,0.98,1451.0,1.451,less than 5km
17,1106234039745058155,16365362,2014-06-04,"Melbourne, Australia",within an hour,100%,100%,f,t,t,...,5.00,5.00,1,0,1,0,1.00,24913.0,24.913,less than 25km
18,1053974246558615906,551937839,2023-12-21,"Point Cook, Australia",within an hour,97%,94%,f,t,t,...,4.00,5.00,7,7,0,0,2.00,6951.0,6.951,less than 10km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25072,30836862,22548036,2014-10-14,"Melbourne, Australia",within a few hours,100%,95%,f,t,t,...,4.78,4.53,3,3,0,0,2.74,3351.0,3.351,less than 5km
25078,1061360537948958353,517641271,2023-06-01,"Melbourne, Australia",within an hour,89%,90%,f,t,t,...,5.00,3.67,11,11,0,0,3.00,2627.0,2.627,less than 5km
25081,710398956041051683,90729398,2016-08-19,"Victoria, Australia",within a few hours,82%,41%,f,t,t,...,5.00,3.00,339,75,262,2,0.08,3196.0,3.196,less than 5km
25082,925945938663212093,516470484,2023-05-25,"Melbourne, Australia",within an hour,100%,93%,t,t,t,...,4.98,4.84,5,5,0,0,6.09,1641.0,1.641,less than 5km


In [96]:
#remove $ sign from column
melb_refined_df["price"] = melb_refined_df["price"].str.replace('$', '').str.replace(',', '')
#convert price column float
#melb_price_review_df.loc[:,"price"] = pd.to_numeric(melb_price_review_df["price"], errors='coerce')
melb_refined_df["price"] = melb_refined_df["price"].astype("float")
melb_refined_df.head()

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distances_from_cbd,distances_from_cbd_km,distances_from_cbd_category
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,4.37,3.97,8,0,4,4,1.21,7434.0,7.434,less than 10km
11,727686708237430703,382295537,2020-12-30,"Melbourne, Australia",within a few hours,89%,57%,f,t,t,...,4.20,4.20,3,0,3,0,0.81,10140.0,10.140,less than 15km
16,1051913801382467131,22860147,2014-10-22,"Melbourne, Australia",within an hour,100%,99%,f,t,t,...,4.50,4.00,105,105,0,0,0.98,1451.0,1.451,less than 5km
17,1106234039745058155,16365362,2014-06-04,"Melbourne, Australia",within an hour,100%,100%,f,t,t,...,5.00,5.00,1,0,1,0,1.00,24913.0,24.913,less than 25km
18,1053974246558615906,551937839,2023-12-21,"Point Cook, Australia",within an hour,97%,94%,f,t,t,...,4.00,5.00,7,7,0,0,2.00,6951.0,6.951,less than 10km


In [97]:
#categorise the price per night of the accomodation
price_categories = []
for index, row in melb_refined_df.iterrows():
    price_cat = row["price"]

    if price_cat < 100:
        category = "less than $100"
    elif price_cat < 200:
        category = "less than $200"  
    elif price_cat < 300:
        category = "less than $300"            
    elif price_cat < 400:
        category = "less than $400"  
    elif price_cat < 500:
        category = "less than $500"
    elif price_cat < 600:
        category = "less than $600"
    elif price_cat < 700:
        category = "less than $700"
    elif price_cat < 800:
        category = "less than $800"
    elif price_cat < 900:
        category = "less than $900"
    elif price_cat < 1000:
        category = "less than $1000"
    else:
        category = "greater than $1000"
        
    price_categories.append(category)

melb_refined_df["Price Categories"] = price_categories
melb_refined_df

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,...,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distances_from_cbd,distances_from_cbd_km,distances_from_cbd_category,Price Categories
4,572431386115618941,446636199,2022-02-24,"St Kilda, Australia",within an hour,95%,99%,f,t,t,...,3.97,8,0,4,4,1.21,7434.0,7.434,less than 10km,less than $200
11,727686708237430703,382295537,2020-12-30,"Melbourne, Australia",within a few hours,89%,57%,f,t,t,...,4.20,3,0,3,0,0.81,10140.0,10.140,less than 15km,less than $100
16,1051913801382467131,22860147,2014-10-22,"Melbourne, Australia",within an hour,100%,99%,f,t,t,...,4.00,105,105,0,0,0.98,1451.0,1.451,less than 5km,less than $200
17,1106234039745058155,16365362,2014-06-04,"Melbourne, Australia",within an hour,100%,100%,f,t,t,...,5.00,1,0,1,0,1.00,24913.0,24.913,less than 25km,less than $200
18,1053974246558615906,551937839,2023-12-21,"Point Cook, Australia",within an hour,97%,94%,f,t,t,...,5.00,7,7,0,0,2.00,6951.0,6.951,less than 10km,less than $300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25072,30836862,22548036,2014-10-14,"Melbourne, Australia",within a few hours,100%,95%,f,t,t,...,4.53,3,3,0,0,2.74,3351.0,3.351,less than 5km,less than $200
25078,1061360537948958353,517641271,2023-06-01,"Melbourne, Australia",within an hour,89%,90%,f,t,t,...,3.67,11,11,0,0,3.00,2627.0,2.627,less than 5km,less than $200
25081,710398956041051683,90729398,2016-08-19,"Victoria, Australia",within a few hours,82%,41%,f,t,t,...,3.00,339,75,262,2,0.08,3196.0,3.196,less than 5km,less than $200
25082,925945938663212093,516470484,2023-05-25,"Melbourne, Australia",within an hour,100%,93%,t,t,t,...,4.84,5,5,0,0,6.09,1641.0,1.641,less than 5km,less than $200


In [98]:
# Creating clean CSV with melbourne data
melb_refined_df.to_csv('Outputs/Melbourne_refined_listings_cleaned.csv', index=False)